In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-fd377f76-7d9a-2cd2-d802-10c3b342bac2)


In [3]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-02-12 12:15:48--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-02-12 12:15:48 (114 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [4]:
import helper_functions as hf

In [5]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2023-02-12 12:18:22--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.99.128, 172.253.117.128, 74.125.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.99.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2023-02-12 12:18:22 (106 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [7]:
hf.unzip_data('nlp_getting_started.zip')

In [8]:
from matplotlib import pyplot as plt

In [9]:
import pandas as pd

In [10]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [11]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [12]:
train_df_shuffled = train_df.sample(frac = 1, random_state=42)

In [13]:
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [14]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [15]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [16]:
len(train_df), len(test_df)

(7613, 3263)

In [17]:
import random

random_index = random.randint(0, len(train_df)-5)

In [18]:
random_index

1672

In [23]:
for row in train_df_shuffled[['text','target']][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f'Target: {target}','(real disaster)' if target > 0 else '(not real disaster)')
  print(f'Text:\n{text}\n')
  print('---\n')


Target: 0 (not real disaster)
Text:
@MsMigot wow what convincing &amp; compelling evidence to change my view that is. climate change deniers I tend to destroy by giving...EVIDENCE

---

Target: 1 (real disaster)
Text:
Hurricane 30STM quem lembra

---

Target: 1 (real disaster)
Text:
Flood and Mudslide Warning for East Fjords | Iceland Review http://t.co/534q3Jg2OV #icelandreview via @iceland_review

---

Target: 1 (real disaster)
Text:
Police Officer Wounded Suspect Dead After Exchanging Shots http://t.co/brE2lGmn7C #ABC #News #AN247

---

Target: 0 (not real disaster)
Text:
Are the safety improvements being made to Route 198 in Buffalo doing more harm than good? http://t.co/rqlPSAYE3B

---



In [24]:
from sklearn.model_selection import train_test_split

In [26]:
X_train,X_test, y_train, y_test = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                   train_df_shuffled['target'].to_numpy(),
                                                   test_size = 0.1,
                                                   random_state = 42)

In [27]:
len(X_train), len(X_test),len(y_train), len(y_test)

(6851, 762, 6851, 762)

In [32]:
X_train[:10]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk',
       '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
       'destroy the free fandom honestly',
       'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
       '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
       'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
      dtype=object)

In [31]:
y_train[:10]

array([0, 0, 1, 0, 0, 1, 1, 0, 1, 1])

In [33]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [41]:
text_vectorizer = TextVectorization(max_tokens=10000,
                                    standardize = 'lower_and_strip_punctuation',
                                    split = 'whitespace',
                                    ngrams = None,
                                    output_mode = 'int',
                                    output_sequence_length = None,
                                    pad_to_max_tokens = True)

In [40]:
round(sum([len(s.split()) for s in X_train])/len(X_train))

15

In [43]:
max_vocab_lenght = 10000
max_lenght = 15

In [44]:
text_vectorizer = TextVectorization(max_tokens=max_vocab_lenght,
                                    output_mode = 'int',
                                    output_sequence_length = max_lenght)

In [45]:
text_vectorizer.adapt(X_train)

In [46]:
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [47]:
random_sentence = random.choice(X_train)

print(f'Original text:\n{random_sentence}\
        \n\nVectorized version:\n{text_vectorizer([random_sentence])}')

Original text:
@Lobo_paranoico Mad Men        

Vectorized version:
[[  1 942 646   0   0   0   0   0   0   0   0   0   0   0   0]]


In [48]:
words_in_vocab = text_vectorizer.get_vocabulary()

In [50]:
words_in_vocab[:5]

['', '[UNK]', 'the', 'a', 'in']

In [51]:
words_in_vocab[-5:]

['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']

In [56]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim = max_vocab_lenght,
                             output_dim = 128,
                             input_length=max_lenght)

In [63]:
random_sentence = random.choice(X_train)
print(f'Original text:\n{random_sentence} \
        \n\nEmbeded version: {embedding(text_vectorizer(random_sentence))}')

Original text:
320 [IR] ICEMOON [AFTERSHOCK] | http://t.co/e14EPzhotH | @djicemoon | #Dubstep #TrapMusic #DnB #EDM #Dance #IcesÛ_ http://t.co/22a9D5DO6q         

Embeded version: [[ 0.01797115  0.04695031  0.02274109 ...  0.0322406   0.04120023
   0.03125354]
 [ 0.02501277 -0.04812377  0.04265347 ... -0.0487326  -0.01076342
  -0.04828067]
 [-0.00515801  0.03119853  0.00207537 ... -0.01392193  0.03501593
  -0.04529643]
 ...
 [-0.0092788  -0.02559779 -0.02250119 ... -0.00611198  0.04559277
   0.01775341]
 [ 0.04423758  0.00299553  0.04465336 ... -0.03095425 -0.00285513
   0.03843465]
 [ 0.04423758  0.00299553  0.04465336 ... -0.03095425 -0.00285513
   0.03843465]]


In [67]:
embedding(text_vectorizer(random_sentence))[0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 0.01797115,  0.04695031,  0.02274109, -0.00906932,  0.02478376,
        0.04829049,  0.03125321, -0.02309763,  0.04804583,  0.0108918 ,
        0.01364941,  0.01678684,  0.0150363 , -0.02400801, -0.02537267,
        0.04333911,  0.02247265, -0.0412771 , -0.01676444,  0.00405927,
        0.04331933,  0.04635186,  0.00424964,  0.00600611,  0.04692681,
        0.03517094, -0.02560434, -0.02882191, -0.01860183, -0.01326514,
        0.01905547, -0.00037   ,  0.04401522,  0.01091594, -0.02517101,
       -0.02999619,  0.00649738, -0.01510736, -0.0012909 , -0.01269459,
       -0.00197762,  0.01647164, -0.04040544,  0.0180567 , -0.02793102,
        0.01625713,  0.02211047,  0.02392845,  0.0072368 , -0.00153565,
       -0.02911394,  0.04229095,  0.04686321,  0.03865052, -0.0374907 ,
       -0.04469572,  0.04226812,  0.03485217, -0.03277223, -0.03882444,
       -0.02556387,  0.02198675, -0.04690231,  0.04353373, -0.01291052,
        0.030753

## Model 0: Naive Bayes

In [69]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
  ('tfidf', TfidfVectorizer()),
  ('clf', MultinomialNB())  
])

model_0.fit(X_train,y_train)



Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [72]:
baseline_score = model_0.score(X_test,y_test)
print(f'Our model achieves an accuracy of: {baseline_score*100:.2f}%')

Our model achieves an accuracy of: 79.27%


In [73]:
baseline_preds = model_0.predict(X_test)

In [74]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true,
                      y_pred):
  """
  Calculates model's accuracy, precision, recall and f1 score of 
  binary classification.
  """

  model_accuracy = accuracy_score(y_true, y_pred)
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true,y_pred, average = 'weighted')
  model_results = {
      'accuracy':model_accuracy,
      'precision':model_precision,
      'recall':model_recall,
      'f1':model_f1
  }

  return model_results

In [76]:
baseline_results = calculate_results(y_test,
                                     baseline_preds)

baseline_results

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}